# Constraint satisfaction problem

In the previous tasks the state was opaque: an algorithm could execute an action and transform a state to another state, but that was it. It could not depend in its search on the details of a state, because the state representation was different from problem to problem. In CSP this is no longer true: the state always consists of an assignment of values over some domains to variables. This enables an algorithm to interact with the state directly, not only through opaque actions. 

Allowed assignments are limited by constraints in the problem and broadly speaking they can be divided into three groups:

* *Unary constraints* each concerning only a single variable (e.g., $x>0$)
* *Binary constraints* relating two variables to each other (e.g., $x>y$)
* *General constraints* concerning more than two variables at once

In general, the shape of constraints is unrestricted and we are not limited to numeric domains and constraints in the form of inequalities. In principle, one could employ arbitrary boolean functions as constraints.

The following class `CSP` is a very limited representation of such a problem capable only of handling:

* finite domains that can be materialized (e.g., the set of all 32-bit integers is doable in theory; the set of all 64-bit integers is out of reach) 
* unary constraints by setting the domain of a variable appropriately;
* binary constraints defined in terms of Python's operators `==`, `!=`, `<`, `<=`, `>` and `>=` such that both sides are variables.

About its methods:

* The constructor (`__init__`) requires no arguments.
* `addVariable` requires two arguments: 
    * `var` - a variable (a string representing its name, but really, it could be any object);
    * `domain` - a collection of the allowed values for the variable
* `addBinaryConstraint` also requires three arguments:
    * `var1` - a variable, the left-hand side of the constraint
    * `op` - an operator (one of the following strings: `'=='`, `'!='`, `'<'`, `'<='`, `'>'`, `'>='`)
    * `var2` - a variable, the right-hand side of the constraint
* `verify` is used to verify whether the relation given by the second argument `op` holds between the values given as the first argument `left` and the third argument `right`. Returns `True` if it does and `False` otherwise.
* `is_complete` gets a single argument: a dictionary representing an assignment of values to variables. From now on, we call such a dictionary an *assignment*. It returns `True` if all the variables of the problem are present in the dictionary, and `False` otherwise.
* `is_consistent` also requries an assignment. It returns `True` if all the values in the assignment are such that they come from the respecitve domains, and no binary constrain is violated (but if the assignment is not complete then some may be unfulfilled).

In [37]:
class CSP:
    def __init__(self):
        self.domains = {}
        self.binary = []
    
    def addVariable(self, var, domain):        
        assert var not in self.domains
        self.domains[var] = set(domain)
        
    def addBinaryConstraint(self, var1, operator, var2):
        assert var1 in self.domains
        assert var2 in self.domains
        c = (var1, operator, var2)
        self.binary.append(c)      
        
    def verify(self, left, op, right):
        if op[0] == '=':
            return left == right
        if op == '!=':
            return left != right
        if op == '<':
            return left < right
        if op == '<=':
            return left <= right
        if op == '>':
            return left > right
        if op == '>=':
            return left >= right
        
    def is_complete(self, assignment):
        return self.domains.keys() <= assignment.keys() 
        
    def is_consistent(self, assignment):
        for var, value in assignment.items():            
            if value not in self.domains[var]:
                return False
        for var1, op, var2 in self.binary:
            if var1 in assignment and var2 in assignment:
                if not self.verify(assignment[var1], op, assignment[var2]):
                    return False
        return True

## Making Australia colorful again

Lets model a map coloring problem. In particular, we'll color the following map of Australia. It can be simplified to a graph presented in the right-hand side of the picture below.

![](aima-images/fig_6_1.png)

We begin by creating a new problem in the variable `australia` over the domain consisting of three values: `'R'` statnding for the color red, `'G'` standing for green and `'B'` standing for blue. Of course the particular symbols used are not relevant. There are 7 teritories, so we add 7 variables to the problem.

In [38]:
australia = CSP()
australia.addVariable('WA', {'R', 'G', 'B'})
australia.addVariable('NT', {'R', 'G', 'B'})
australia.addVariable('SA', {'R', 'G', 'B'})
australia.addVariable('Q', {'R', 'G', 'B'})
australia.addVariable('NSW', {'R', 'G', 'B'})
australia.addVariable('V', {'R', 'G', 'B'})
australia.addVariable('T', {'R', 'G', 'B'})

Any two teritories of Australia sharing a border should be colored using different colours. We thus define 9 binary constraints.

In [39]:
australia.addBinaryConstraint('WA', '!=', 'NT')
australia.addBinaryConstraint('WA', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'NSW')
australia.addBinaryConstraint('SA', '!=', 'V')
australia.addBinaryConstraint('Q', '!=', 'NSW')
australia.addBinaryConstraint('NSW', '!=', 'V')
print(australia.binary)

[('WA', '!=', 'NT'), ('WA', '!=', 'SA'), ('NT', '!=', 'SA'), ('NT', '!=', 'Q'), ('SA', '!=', 'Q'), ('SA', '!=', 'NSW'), ('SA', '!=', 'V'), ('Q', '!=', 'NSW'), ('NSW', '!=', 'V')]


Let's see how the `verify` method works.

In [40]:
print("Is G=G?", australia.verify('G', '==', 'G'))
print("Is 5<7?", australia.verify(5, '<', 7))
print("Is 5>=7?", australia.verify(5, '>=', 7))

Is G=G? True
Is 5<7? True
Is 5>=7? False


Lets now consider an assignment such that *Western Australia* is red and the rest is unspecified. Is it a consistent assignment?

In [41]:
australia.is_consistent({'WA': 'R'})

True

Is it a complete assignment?

In [42]:
australia.is_complete({'WA': 'R'})

False

As expected it didn't work: there are still 9 unfulfiled constraints. So lets extend the solution.

In [43]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? False


Now it almost works: all the constraints are satisfied, but the solution is still not complete, as Tasmania is still colorless.

In [44]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G', 'T': 'B'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? True


Finally, we have manually collored the map of Australia! Now let's make computer do this for us.

## Task 1: Backtracking

Complete the following cell of code by implementing the recursive backtracking algorithm. Basically, it is a recursive DFS where an action is assigning a value to a variable. Avoid making copies of assignments, instead operate on a single assignment, and update it accordingly. Increment `self.counter` every time you check wheter an assigment is a solution (i.e., is consistent and complete). Use the provided methods `select_unassigned_variable` and `order_domain_values` to, respectively, get a next variable to assign a value to, and to get the list of allowed values for the variable. `solve` should return a solution (i.e., a complete, consistent assignment) or `None`.

In [45]:
class RecursiveBacktracking:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
        
    def select_unassigned_variable(self, assignment):
        for var in self.csp.domains.keys():
            if var not in assignment:
                return var
        return None
    
    def order_domain_values(self, variable, assignment):
        return self.csp.domains[variable]
        
    
    def solve(self):
        self.counter = 0
        assignment = {}
        #visited_assignments = set()
        
        return self.solve_recur(assignment)
        
    def solve_recur(self, assignment):
#         self.counter += 1
        if self.csp.is_complete(assignment):
            return assignment
        #print("\nAssignment: ", len(assignment))
        key = self.select_unassigned_variable(assignment)
        domain = self.order_domain_values(key, assignment)
        #print(f"Variable to assign: {key} which has domain: {domain}")
        for val in domain:
            #print(f"Variable to assign: {key}, value to assign: {val}")
            if val:
                """avoids errors when there would not be values in the domain"""
                assignment[key] = val
                #print("new assignment: ", len(assignment))

                self.counter += 1
                if self.csp.is_consistent(assignment):
                    #print("new assignment is consistent")
                    #visited_assignments.add(tuple(assignment))
                    new_assignment = self.solve_recur(assignment)

                    if new_assignment:
                        return new_assignment
                    
                    if assignment:
                        assignment.popitem()
            #print(f"\t{val} was not a right value for {key}")
            
        return None

Let's test it on the Australia problem! In the cell below you should get a consistent, complete assignment, i.e., a solution to the problem.

In [46]:
solver = RecursiveBacktracking(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'WA': 'R', 'NT': 'G', 'SA': 'B', 'Q': 'R', 'NSW': 'G', 'V': 'R', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 11


## Task 2: Extend `RecursiveBacktracking` with the MRV and degree heuristics

Modify your `RecursiveBacktracking` to include the MRV (minimum remaining values) heuristics and the degree heuristics. Implement them in the appropriate places below in the class `RecursiveBacktrackingWithHeuristics`. If you want, you can also implement the least constraining value heuristics, but this is not mandatory. Follow the same assumptions as for `RecursiveBacktracking`, so expose the main backtracking algorithm as the `solve` and keep track of the number of considered assignments in the `counter` variable.

In [47]:
import math
import operator

In [48]:
class RecursiveBacktrackingWithHeuristics(RecursiveBacktracking):
    
    def get_constrains_with_given_variable(self, variable):
        return list(filter(lambda constraint: constraint[0] == variable or constraint[2] == variable,
                                           self.csp.binary))
    
    def get_possible_values_and_constraints(self, variable, assignment):
        domain = self.csp.domains[variable]
        
        constrains_for_var = self.get_constrains_with_given_variable(variable)
#         print(assignment)
        for var1, op, var2 in constrains_for_var:
#             print(var1, op, var2)
            new_domain = set(domain)
            if var1 == variable and var2 in assignment:
#                 print("var2 in assignment")
                val_in_assignment = assignment[var2]
#                 print(domain, op, val_in_assignment)
                #domain = filter(lambda val: self.csp.verify(val, op, val_in_assignment), domain)
                #I've really tried to make the filter work but I resign in the end
                domain = {val if self.csp.verify(val_in_assignment, op, val) else None for val in domain}
                
            elif var1 in assignment:
#                 print("var1 in assignment")
                val_in_assignment = assignment[var1]
#                 print(domain, op, val_in_assignment)
#                 domain = filter(lambda val: self.csp.verify(val_in_assignment, op, var2), domain)
                domain = {val if self.csp.verify(val_in_assignment, op, val) else None for val in domain}

#             else:
#                 print("no variable in assignment")

#             print("Changed domain: ",new_domain)
        
#         print("Final domain: ", domain)
        try:
            domain.remove(None)
        except:
            pass
        return (domain, constrains_for_var) 

        
    def select_unassigned_variable(self, assignment):
        #this MRV is not efficient at all but this is because I tried to avoid forward propagation
        #with forward propagation it would be just enough to count number of values in domain of given variable 
        to_assign = dict()
        min_remaining_val = math.inf
        for variable in self.csp.domains.keys():
            if variable not in assignment:
                #assigning number of possible values for a given variable
                domain, constrains_for_var = self.get_possible_values_and_constraints(variable, assignment)
                
                if not domain:
                    return variable
                
                num_of_constraints = len(constrains_for_var)
                num_of_values = len(domain)
                min_remaining_val = min(num_of_values, min_remaining_val)
                
                if num_of_values <= min_remaining_val:
                    to_assign[variable] = (num_of_values, num_of_constraints)
        
#         print(min_remaining_val, to_assign)
        #filtering out those variables which has not the min number of feasible values
        vars_to_consider = list(filter(lambda var: to_assign[var][0]  == min_remaining_val, to_assign))
        #setting a variable as this with maximal number of constraints on other variables
        variable = max(vars_to_consider, key=lambda var: to_assign[var][1])
        return variable
    
    def order_domain_values(self, variable, assignment):
        domain, constaints = self.get_possible_values_and_constraints(variable, assignment)
        
        return domain
    

Lets test your implementation by solving the problem of coloring Australia!

In [49]:
solver = RecursiveBacktrackingWithHeuristics(australia)
assignment = solver.solve()

print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'R', 'NT': 'B', 'Q': 'G', 'NSW': 'B', 'WA': 'G', 'V': 'G', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 7


**Compare the number of considered assignments for `RecursiveBacktracking` and `RecursiveBacktrackingWithHeuristics`. How do they differ? Why do you think is that?**

The recursive backtracking with heuristic has figured out the proper assignment within the first try, so it is definately better than this one without heuristic. The reason why it happened is quite simple. Firstival, the variables are assigned such that those which has biggest priority (least number of acceptable values) goes first. Secondly, in `order_domain_values` method I consider just acceptable values.

## Sudoku!

Coloring Australia is really a bit boring. Lets deal with something more exciting: SUDOKU! I assume you know the rules, but if you don't, go look them up, spend some time solving a few puzzles and familiarize yourself with the puzzle.
The cell below defines a string `puzzle` which represents a Sudoku puzzle from the picture below. A digit in `puzzle` corresponds to a digit in the puzzle, while an underscore `_` corresponds to a blank cell in the puzzle. For your convenience there's also variable `solution` containing the correct solution to this particular puzzle.

![](aima-images/fig_6_4.png)

In [50]:
# puzzle = '''
# __3_2_6__
# 9__3_5__1
# __18_64__
# __81_29__
# 7_______8
# __67_82__
# __26_95__
# 8__2_3__9
# __5_1_3__
# '''
puzzle = '''
8________
__36_____
_7__9_2__
_5___7___
____457__
___1___3_
__1____68
__85___1_
_9____4__
'''

In [51]:
solution = '''
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382
'''

## Task 3: Sudoku as CSP

Using the string `puzzle` defined above write in the cell below code that will add all the necessary constraints to the CSP represented by the variable `sudoku`. If you can and want, write a parser that will construct the CSP from a string representation of the puzzle. Otherwise, you can construct the problem manually.

In [52]:
sudoku = CSP()

def assignment_to_string(assignment):
    puzzle_map = {(x, y):0 for x in range(9) for y in range(9)}
    result = []
    for key, val in assignment.items():
        puzzle_map[key] = val
    for idx, (key, val) in enumerate(puzzle_map.items()):
        if not (idx)%9:
            result.append('\n')
        result.append(str(val))
    result.append('\n')
    return ''.join(result)

def check_solution(assignment_string, solution):
    res = []
    for idx in range(len(solution)):
        if assignment_string[idx] == solution[idx]:
            res.append(assignment_string[idx])
        else:
            print(f"Differs at idx: {idx}, in assignment: {assignment_string[idx]}, in solution{solution[idx]}")
            print(''.join(res))
            return False
    return True

def parser(csp, puzzle):
    puzzle_map = {(x, y): set(range(1,10)) for x in range(9) for y in range(9)}
    formatted_puzzle = puzzle.split()
    
    for pos in puzzle_map:
        row, col = pos
#         print(pos)
        if formatted_puzzle[row][col] != '_':
            val = int(formatted_puzzle[row][col])
            puzzle_map[pos] = {val}
            
            #deleting value for whole row
            for c in range(9):
                if c != col:
                    try:
                        puzzle_map[(row,c)].remove(val)
                    except:
    #                     print("You've tried to remove value from position at which value is predefined by puzzle")
                        pass
            
            #deleting value in whole column
            for r in range(9):
                if r != row:
                    try:
                        puzzle_map[(r,col)].remove(val)
                    except:
    #                     print(f"You've tried to remove value which is given at pos: ({r}, {col}) with " )
                        pass
            
            #deleting value in the squarefor 
            for r in range(row-row%3, row-row%3 + 3):
                for c in range(col - col%3, col - col%3 + 3):
                    if r != row and c != col:
                        try:
                            puzzle_map[(r, c)].remove(val)
                        except:
                            #removing value which is none type
                            pass
    
    constraints = set()
    for pos in puzzle_map:
        
        row, col = pos
        op = '!='
            
        #constraints for row
        for c in range(9):
            if c != col and (pos, op, (row, c)) not in constraints and ((row,c), op, pos) not in constraints:
                constraints.add((pos, op, (row, c)))

        #constraints for column
        for r in range(9):
            if r != row and (pos, op, (r, col)) not in constraints and ((r,col), op, pos) not in constraints:
                constraints.add((pos, op, (r, col)))

        #deleting value in the squarefor 
        for r in range(row-row%3, row-row%3 + 3):
            for c in range(col - col%3, col - col%3 + 3):
                if r != row and c != col:
                    if (pos, op, (r, c)) not in constraints and ((r,c), op, pos) not in constraints:
                        constraints.add((pos, op, (r, c)))

    for pos, domain in puzzle_map.items():
        sudoku.addVariable(pos, domain)
    
    for var1, op, var2 in constraints:
        sudoku.addBinaryConstraint(var1, op, var2)
                        
    return puzzle_map

puzzle_map = parser(sudoku, puzzle)
# formatted_puzzle = puzzle.split()
# for key, value in puzzle_map.items():
#     print(f"Key: {key}, value in puzzle: {formatted_puzzle[key[0]][key[1]]} domain: {value}")

Let's see if it works... Unless you did some clever tricks and/or are quite patient - it does not, it is too slow.

In [53]:
solver = RecursiveBacktrackingWithHeuristics(sudoku)
%time assignment = solver.solve()
print("Assignment:", assignment_to_string(assignment))
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter) 

CPU times: user 6.01 s, sys: 6.65 ms, total: 6.02 s
Wall time: 6.01 s
Assignment: 
812753649
943682175
675491283
154237896
369845721
287169534
521974368
438526917
796318452

Is consistent? True
Is complete? True
# considered assignments 1394


## Task 4: Solving sudoku

In order to make it working, we need to extend backtracking with inference. Complete the following class `RecursiveBacktrackingWithAC3` in such a way that it uses the same heuristics as `RecursiveBacktrackingWithHeuristics` and on top of that uses the AC3 algorithm after a new value is added to the assignment.

In this task, you don't need to stick to the previous API with method. A constructor with a single argument, and an argumentless `solve` are sufficient, heuristics and AC3 can be handled however you want.

In [54]:
import copy
from collections import deque

In [55]:
class RecursiveBacktrackingWithAC3(RecursiveBacktrackingWithHeuristics):
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
        
    def AC3(self, domains):
        constraints = deque(self.csp.binary)
        while constraints:
            constraint = constraints.popleft()
            var1, op, var2 = constraint
            
            if self.remove_incosistent_values(constraint, domains):
                constraints_with_var1 = self.get_constrains_with_given_variable(var1)
                
                for constraint_to_reconsider in constraints_with_var1:
                    constraints.append(constraint_to_reconsider)
        
    def remove_incosistent_values(self, constraint, domains):
        removed = False
        var1, op, var2 = constraint
        domain1 = domains[var1]
        domain2 = domains[var2]
        values_to_remove = []
        for value1 in domain1:
            allowing_values = 0
            for value2 in domain2:
                
#                 print(f"Constraint: {constraint}; expression {value1} {op} {value2}; eval: {self.csp.verify(value1, op, value2)}")
                if not allowing_values and self.csp.verify(value1, op, value2):
                    allowing_values+= 1
            if not allowing_values:
                values_to_remove.append(value1)
        
#         print(f"Variables to remove: {values_to_remove}")
        if values_to_remove:
            removed = True
            for value in values_to_remove:
                domain1.remove(value)
        
        domains[var1] = domain1
        domain1 = domains[var1]
        values_to_remove = []
        
        for value2 in domain2:
            allowing_values = 0
            for value1 in domain1:
                
#                 print(f"Constraint: {constraint}; expression {value1} {op} {value2}; eval: {self.csp.verify(value1, op, value2)}")
                if not allowing_values and self.csp.verify(value1, op, value2):
                    allowing_values+= 1
            if not allowing_values:
                values_to_remove.append(value2)
        
#         print(f"Variables to remove: {values_to_remove}")
        if values_to_remove:
            removed = True
            for value in values_to_remove:
                domain2.remove(value)
        
        domains[var2] = domain2
                
        return removed
    
    def select_unassigned_variable(self, assignment, domains):
        variable = None
        lengths_of_domains = {var:len(domain) for var, domain in domains.items() if var not in assignment}
        min_domain_lenght = min(lengths_of_domains.values())
        
        variables_to_consider = list(filter(lambda x: x[1] == min_domain_lenght, lengths_of_domains.items()))
        
        if len(variables_to_consider) > 1 and min_domain_lenght > 0:
            variable = max(variables_to_consider, 
                           key=lambda var: len(self.get_constrains_with_given_variable(var[0])))
        else:
            variable = variables_to_consider.pop()
        
        return variable[0]
    
    def order_domain_values(self, var, domains):
        return domains[var]
    
    def forward_check(self, assignment, var, given_val, domains):
        constraints = self.get_constrains_with_given_variable(var)
        
        for constraint in constraints:
            var1, op, var2 = constraint
            inconsistent_vals = set()
#             print("\tDomain before forward check: ", end='')
            
            if var1 == var and var2 not in assignment:
#                 print(domains[var2], " first", var2)
                for value in domains[var2]:
                    if not self.csp.verify(given_val, op, value):
                        inconsistent_vals.add(value)
                domains[var2] -= inconsistent_vals
#                 print(f"\tDomain after check: {domains[var2]}")
                
            elif var1 not in assignment:
#                 print(domains[var1], " second", var1)
                for value in domains[var1]:
                    if not self.csp.verify(value, op, given_val):
                        inconsistent_vals.add(value)
                domains[var1] -= inconsistent_vals
#                 print(f"\tDomain after check: {domains[var1]}")
#             print(f"\tIncosistent values: {inconsistent_vals}")
        
        
    def solve(self):
        self.counter = 0
        assignment = {}
        #visited_assignments = set()
        
        return self.solve_recur(assignment, copy.deepcopy(self.csp.domains))
        
    def solve_recur(self, assignment, domains):
#         self.counter += 1
        if self.csp.is_complete(assignment):
            return assignment
        self.AC3(domains)
#         print("\nDomains: ", domains)
#         print("Assignment: ", len(assignment))
        key = self.select_unassigned_variable(assignment, domains)
        domain = self.order_domain_values(key, domains)
#         print(f"Variable to assign: {key} which has domain: {domain}")
        for val in domain:
#             print(f"Variable to assign: {key}, value to assign: {val}")
            if val:
                """avoids errors when there would not be values in the domain"""
                assignment[key] = val
#                 print("new assignment: ", len(assignment))

                self.counter += 1
                if self.csp.is_consistent(assignment):
#                     print("new assignment is consistent")
                    new_domains = copy.deepcopy(domains)
                    new_domains[key] = {val}
                    self.forward_check(assignment, key, val, new_domains)
#                     print("Domains after forward check: ", new_domains)
                    new_assignment = self.solve_recur(assignment, new_domains)

                    if new_assignment:
                        return new_assignment
            
                    if assignment:
                        assignment.pop(key, None)
#             print(f"\t{val} was not a right value for {key}")
            
        return None

Let's test it on both considered problem. If everything went all right, both cells should terminate within few seconds at most, and you should see that the number of considered assignments is very small.

In [56]:
solver = RecursiveBacktrackingWithAC3(australia)
%time assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

CPU times: user 721 µs, sys: 11 µs, total: 732 µs
Wall time: 750 µs
Assignment {'SA': 'R', 'NT': 'G', 'Q': 'B', 'NSW': 'G', 'WA': 'B', 'V': 'B', 'T': 'R'}
Is consistent? True
Is complete? True
# considered assignments 7


In [57]:
solver = RecursiveBacktrackingWithAC3(sudoku)
%time assignment = solver.solve()
print("Assignment", assignment_to_string(assignment))
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter)

CPU times: user 3.82 s, sys: 3.33 ms, total: 3.83 s
Wall time: 3.83 s
Assignment 
812753649
943682175
675491283
154237896
369845721
287169534
521974368
438526917
796318452

Is consistent? True
Is complete? True
# considered assignments 651


------------
The pictures are from "Artificial Intelligence: A Modern Approach" 3rd ed.